In [3]:
import pandas as pd
nba = pd.read_csv('nba_2013.csv')
print(nba.columns.values)

['player' 'pos' 'age' 'bref_team_id' 'g' 'gs' 'mp' 'fg' 'fga' 'fg.' 'x3p'
 'x3pa' 'x3p.' 'x2p' 'x2pa' 'x2p.' 'efg.' 'ft' 'fta' 'ft.' 'orb' 'drb'
 'trb' 'ast' 'stl' 'blk' 'tov' 'pf' 'pts' 'season' 'season_end']


# 1: Finding Similar Rows With Euclidean Distance
Instructions
Create a function for calculating the Euclidean distance between two pandas series objects.
Use the function to find the Euclidean distance between selected_player and each row in nba.
Use the .apply(func, axis=1) method on dataframes to apply function func to each row.
The function should take row as its first argument.
Only use the columns in distance_columns to compute the distance. - See the documentation on the apply() method if you get stuck.
Assign the resulting pandas series to lebron_distance.
Hint
selected_player and the row you pass into the Euclidean distance function will both be pandas series.
You can return a column value by indexing the series by name.
row["fg"] will return the number of field goals if row is a series.

In [4]:
selected_player = nba[nba["player"] == "LeBron James"].iloc[0]

In [5]:
distance_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg', 'x3p', 'x3pa', 'x3p', 'x2p', 
                    'x2pa', 'x2p', 'efg.', 'ft', 'fta', 'ft.', 'orb', 'drb', 'trb', 'ast', 
                    'stl', 'blk', 'tov', 'pf', 'pts']

In [6]:
import math

def euclidean_distance(row):
    inner_value = 0
    for k in distance_columns:
        inner_value += (row[k] - selected_player[k]) ** 2
        return math.sqrt(inner_value)
    
lebron_distance = nba.apply(euclidean_distance, axis=1)

# 2: Normalizing Columns

# Instructions
Normalize the columns in nba_numeric.
Using .mean() on a dataframe will return the mean of each column.
Using .std() will return the standard deviation of each column.
Assign the result to nba_normalized.
Hint
Subtracting nba_numeric.mean() from nba_numeric will subtract the mean from each column. You can follow the same process to divide by standard deviation.

In [7]:
nba_numeric = nba[distance_columns]
print(nba_numeric)

     age   g  gs    mp   fg   fga   fg  x3p  x3pa  x3p  ...     ft.  orb  drb  \
0     23  63   0   847   66   141   66    4    15    4  ...   0.660   72  144   
1     20  81  20  1197   93   185   93    0     0    0  ...   0.581  142  190   
2     27  53  12   961  143   275  143    0     0    0  ...   0.639  102  204   
3     28  73  73  2552  464  1011  464  128   300  128  ...   0.815   32  230   
4     25  56  30   951  136   249  136    0     1    0  ...   0.836   94  183   
5     25  46   2   330   33    61   33    0     0    0  ...   0.867   37   92   
6     28  69  69  2498  652  1423  652    3    15    3  ...   0.822  166  599   
7     24  65   2  1072  134   300  134    2    13    2  ...   0.660  119  192   
8     38  73   9  1936  240   543  240  116   309  116  ...   0.905   23  182   
9     32  55  28  1278  204   413  204   11    47   11  ...   0.628   79  129   
10    23  80  65  2045  234   494  234   13    48   13  ...   0.664  129  367   
11    31  19   0   185   16 

In [ ]:
nba_normalized = (nba_numeric - nba_numeric.mean()) / nba_numeric.std()

# 3: Finding The Nearest Neighbor

# Instructions
Find the player who's most similar to LeBron James by our distance metric.
You can accomplish this by finding the second lowest value in the euclidean_distances series (the lowest value will correspond to Lebron, as he is most similar to himself), and then cross-referencing the NBA dataframe with the same index.
Assign the name of the player to most_similar_to_lebron.
Hint
You can make a new dataframe using distance_frame = pandas.DataFrame(data={"dist": euclidean_distances, "idx": euclidean_distances.index}).
You can sort the dataframe on a column using sort_values("column_name").
After sorting, it's a matter of picking the second row, finding the idx column value, and using that value to get the correct row in the nba dataframe.

In [ ]:
from scipy.spatial import distance

nba_normalized.fillna(0, inplace=True)

lebron_normalized = nba_normalized[nba["player"] == "LeBron James"]

euclidean_distances = nba_normalized.apply(lambda row: distance.euclidean(row, lebron_normalized), axis=1)

distance_frame = pandas.DataFrame(data={"dist": euclidean_distances, "idx": euclidean_distances.index})

distance_frame.sort_values("dist", inplace=True)

second_smallest = distance_frame.iloc[1]["idx"]

most_similar_to_lebron = nba.loc[int(second_smallest)]["player"]

# 4: Generating Training And Testing Sets

In [9]:
import random
from numpy.random import permutation

random_indices = permutation(nba.index)

test_cutoff = math.floor(len(nba)/3)

test = nba.loc[random_indices[1:test_cutoff]]
train = nba.loc[random_indices[test_cutoff]]

# 5: Generating Training And Testing Sets

In [10]:
x_columns = ['age', 'g', 'gs', 'mp', 'fg', 'fga', 'fg.', 'x3p',
             'x3pa', 'x3p.', 'x2p', 'x2pa', 'x2p.', 'efg.', 'ft', 'fta',
             'ft.', 'orb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf']

In [ ]:
y_column = ["pts"]

from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(train[x_columns], train[y_column])
predictions = knn.predict(test[x_columns])

# 6: Computing Error

# Instructions
Compute the mean squared error between actual and predictions.
Assign the result to mse.
Hint
Follow the formula. You can subtract all of the values in predictions and actual using predictions - actual.

In [ ]:
actual = test[y_column]

mse = (((predictions - actual)**2).sum()) / len(predictions)